# Trabalho final AEDII
- Pegada hídrica todal dos municípios

## Bibliotecas

In [ ]:
!pip install pandas
!pip install missingno
!pip install altair
!pip install openpyxl
!pip install xlrd

In [1]:
import pandas as pd
import missingno as msno
import altair as alt
import os

## Pegada hídrica dos municípios

In [2]:
# Get the current directory
current_dir = os.getcwd()

# Specify the file path relative to the current directory
culturas_mun = os.path.join(current_dir, 'culturas_mun.csv')
pegada_hidrica = os.path.join(current_dir, 'pegada_hidrica.csv')

# Load the CSV file into a pandas DataFrame
dfprod = pd.read_csv(culturas_mun)
dfpeg = pd.read_csv(pegada_hidrica)


In [5]:
dfprod.head()

,Cód.,Município,Abacate,Abacaxi*,Açaí,Algodão arbóreo (em caroço),Algodão herbáceo (em caroço),Alho,Amendoim (em casca),Arroz (em casca),...,Tangerina,Tomate,Trigo (em grão),Triticale (em grão),Tungue (fruto seco),Urucum (semente),Uva,Total,80%,Principal
0,1100015,Alta Floresta D'Oeste (RO),0,100,20,0,0,0,8,405,...,4,1750,0,0,0,0,0,49803,"39842,4",Café (em grão) Total
1,1100023,Ariquemes (RO),0,416,50,0,0,0,0,3120,...,0,67,0,0,0,12,0,54517,"43613,6",Soja (em grão)
2,1100031,Cabixi (RO),0,55,0,0,0,0,1,4320,...,0,0,0,0,0,100,0,210660,168528,Soja (em grão)
3,1100049,Cacoal (RO),0,1955,0,0,0,0,5,24,...,26,350,0,0,0,16,0,46367,"37093,6",Café (em grão) Total
4,1100056,Cerejeiras (RO),0,505,0,0,0,0,6,748,...,0,50,0,0,0,18,0,273323,"218658,4",Milho (em grão)


In [3]:
# Colunas a serem ignoradas durante o melt
colunas_ignoradas = ['Total', '80%', 'Principal']

# Remover as colunas a serem ignoradas
dfprod = dfprod.drop(colunas_ignoradas, axis=1)

# Colunas que serão mantidas como identificadores durante o melt
id_vars = ['Cód.', 'Município']

# Derreter o DataFrame
dfprod = pd.melt(dfprod, id_vars=id_vars, var_name='Produto', value_name='Producao')

# Exibir o DataFrame derretido
dfprod.head()

,Cód.,Município,Produto,Producao
0,1100015,Alta Floresta D'Oeste (RO),Abacate,0
1,1100023,Ariquemes (RO),Abacate,0
2,1100031,Cabixi (RO),Abacate,0
3,1100049,Cacoal (RO),Abacate,0
4,1100056,Cerejeiras (RO),Abacate,0


In [7]:
dfpeg.head()

,nome_cultura,pegada_hidrica
0,Abacate,"491,10"
1,Abacaxi*,"127,40"
2,Açaí,"0,00"
3,Algodão arbóreo (em caroço),"1574,44"
4,Algodão herbáceo (em caroço),"1574,44"


In [4]:
dfpeg['pegada_hidrica'] = dfpeg['pegada_hidrica'].str.replace(',', '.').astype(float)


In [5]:
# Colunas que você deseja manter em dfprodpeg durante o merge
id_vars = ['Cód.', 'Município', 'Produto', 'Producao']

# Merge dos DataFrames usando a coluna 'Produto'
dfprodpeg = pd.merge(dfprod[id_vars], dfpeg[['nome_cultura', 'pegada_hidrica']], left_on='Produto', right_on='nome_cultura', how='left')

# Criar uma nova coluna multiplicando a produção pelo valor da pegada hídrica
dfprodpeg['Pegada_Total'] = dfprod['Producao'] * dfprodpeg['pegada_hidrica']

# Remover a coluna 'nome_cultura', se desejar
dfprodpeg = dfprodpeg.drop(columns=['nome_cultura'])

# Exibir o DataFrame resultante
dfprodpeg.sort_values(by='Pegada_Total', ascending=False).head(10)

,Cód.,Município,Produto,Producao,pegada_hidrica,Pegada_Total
272324,5107925,Sorriso (MT),Milho (em grão),3787800,1137.25,4.307676e+09
316828,5107925,Sorriso (MT),Soja (em grão),2118690,1827.13,3.871122e+09
317041,5218805,Rio Verde (GO),Soja (em grão),1638000,1827.13,2.992839e+09
313488,2911105,Formosa do Rio Preto (BA),Soja (em grão),1576620,1827.13,2.880690e+09
313705,2928901,São Desidério (BA),Soja (em grão),1434027,1827.13,2.620154e+09
316724,5102637,Campo Novo do Parecis (MT),Soja (em grão),1396080,1827.13,2.550820e+09
316782,5106240,Nova Ubiratã (MT),Soja (em grão),1357200,1827.13,2.479781e+09
272278,5106240,Nova Ubiratã (MT),Milho (em grão),2144880,1137.25,2.439265e+09
316743,5103502,Diamantino (MT),Soja (em grão),1286747,1827.13,2.351054e+09
316780,5106224,Nova Mutum (MT),Soja (em grão),1276752,1827.13,2.332792e+09


In [6]:
dfprodpeg_produto = dfprodpeg.groupby('Produto')['Pegada_Total'].sum().sort_values(ascending=False)
dfprodpeg_produto.head()


Produto
Soja (em grão)          2.205365e+11
Milho (em grão)         1.244387e+11
Cana-de-açúcar          8.625041e+10
Café (em grão) Total    1.939466e+10
Arroz (em casca)        1.401583e+10
Name: Pegada_Total, dtype: float64

In [75]:
# Sort the dataframe by 'Pegada_Total' in descending order and select the top 30 values
top_20_df = dfprodpeg_produto.sort_values(ascending=False).head(20)

# Create a bar chart using Altair
bar_chart = alt.Chart(top_20_df.reset_index()).mark_bar().encode(
    x=alt.X('Produto:N', sort='-y'),
    y='Pegada_Total:Q'
).properties(
    width=800,
    height=600
).interactive()

# Display the bar chart
bar_chart


alt.Chart(...)

In [7]:
dfprodpegsum = dfprodpeg.groupby(['Cód.', 'Município'])['Pegada_Total'].sum().reset_index()
dfprodpegsum.head()

,Cód.,Município,Pegada_Total
0,1100015,Alta Floresta D'Oeste (RO),1.686278e+08
1,1100023,Ariquemes (RO),8.744078e+07
2,1100031,Cabixi (RO),3.141676e+08
3,1100049,Cacoal (RO),1.157831e+08
4,1100056,Cerejeiras (RO),3.980391e+08


In [8]:
dfprodpegsum = dfprodpegsum.sort_values(by='Pegada_Total', ascending=False).reset_index(drop=True)
dfprodpegsum.head()

,Cód.,Município,Pegada_Total
0,5107925,Sorriso (MT),8.656694e+09
1,5218805,Rio Verde (GO),5.704177e+09
2,5102637,Campo Novo do Parecis (MT),5.378446e+09
3,5106240,Nova Ubiratã (MT),5.228788e+09
4,5106224,Nova Mutum (MT),4.771651e+09


In [9]:
dfprodpegsum = dfprodpegsum[dfprodpegsum['Pegada_Total'] > 0]
dfprodpegsum.shape


(5498, 3)

In [17]:
dfprodpegsum.to_csv('dfprodpegsum.csv', index=False)

## Segurança hídrica
- Dados: https://metadados.snirh.gov.br/geonetwork/srv/eng/catalog.search#/metadata/c349dc5a-0c01-4f14-9519-e3340fef2c66

In [10]:
# Specify the file path
file_path = 'ISH_Humano_PNSH_ATLAS_anexoNT.xlsx'

# Load the Excel file into a pandas DataFrame, skipping the first row
dfish = pd.read_excel(file_path, skiprows=1)

# Display the DataFrame
dfish.head()


,Código IBGE,Município,UF,REGIAO,RIDE/RM,População em Risco Inerente - PNSH - 2017,População em Risco Pós-Défict - PNSH - 2017,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,...,População em Risco Inerente - Atlas - 2020,População em Risco Pós-Défict - Atlas - 2020,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Inerente - Atlas - 2035,População em Risco Pós-Défict - Atlas - 2035,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035
0,3550308,São Paulo,SP,Sudeste,RM São Paulo,866799,9453272.0,10320071.0,12081082,Mínima,...,1616686,0,1616686,12446934,Média,1867755,3022248,4890003,13118139,Baixa
1,3304557,Rio de Janeiro,RJ,Sudeste,RM Rio de Janeiro,126446,6140929.0,6267375.0,6520266,Mínima,...,230004,668109,898113,6732085,Média,240124,704821,944945,7000802,Média
2,2927408,Salvador,BA,Nordeste,RM Salvador,208802,1076358.0,1285160.0,2953376,Baixa,...,352368,1166166,1518534,2893879,Baixa,419082,1209506,1628588,3150651,Baixa
3,5300108,Brasília,DF,Centro-Oeste,RIDE Distrito Federal,364233,774333.0,1138566.0,2952650,Baixa,...,459008,34028,493036,3064465,Média,668389,49600,717989,3477466,Baixa
4,2304400,Fortaleza,CE,Nordeste,RM Fortaleza,638400,0.0,638400.0,2627482,Baixa,...,770874,380101,1150975,2692722,Baixa,811335,407800,1219135,2841806,Baixa


In [24]:
dfish.columns

Index(['Código IBGE', 'Município', 'UF', 'REGIAO', 'RIDE/RM',
       'População em Risco Inerente - PNSH - 2017',
       'População em Risco Pós-Défict - PNSH - 2017',
       'População em Risco Total - PNSH - 2017', 'População Urbana 2017',
       'ISH Humano - PNSH - 2017', 'População em Risco Inerente - PNSH - 2035',
       'População em Risco Pós-Défict - PNSH - 2035',
       'População em Risco Total - PNSH - 2035', 'População Urbana 2035',
       'ISH Humano - PNSH - 2035',
       'População em Risco Inerente - Atlas - 2020',
       'População em Risco Pós-Défict - Atlas - 2020',
       'População em Risco Total - Atlas - 2020', 'População Urbana 2020',
       'ISH Humano - Atlas - 2020',
       'População em Risco Inerente - Atlas - 2035',
       'População em Risco Pós-Défict - Atlas - 2035',
       'População em Risco Total - Atlas - 2035', 'População Urbana 2035.1',
       'ISH Humano - Atlas - 2035'],
      dtype='object')

In [11]:
colunas = ['Código IBGE', 'Município', 'UF', 'REGIAO', 
       'População em Risco Total - PNSH - 2017', 'População Urbana 2017',
       'ISH Humano - PNSH - 2017', 
       'População em Risco Total - PNSH - 2035', 'População Urbana 2035',
       'ISH Humano - PNSH - 2035',
       'População em Risco Total - Atlas - 2020', 'População Urbana 2020',
       'ISH Humano - Atlas - 2020',
       'População em Risco Total - Atlas - 2035', 'População Urbana 2035.1',
       'ISH Humano - Atlas - 2035']

dfish = dfish[colunas]
dfish.head()

,Código IBGE,Município,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035
0,3550308,São Paulo,SP,Sudeste,10320071.0,12081082,Mínima,11771472.0,13536837,Mínima,1616686,12446934,Média,4890003,13118139,Baixa
1,3304557,Rio de Janeiro,RJ,Sudeste,6267375.0,6520266,Mínima,6358858.0,6606406,Mínima,898113,6732085,Média,944945,7000802,Média
2,2927408,Salvador,BA,Nordeste,1285160.0,2953376,Baixa,1509806.0,3326869,Baixa,1518534,2893879,Baixa,1628588,3150651,Baixa
3,5300108,Brasília,DF,Centro-Oeste,1138566.0,2952650,Baixa,1904114.0,4111833,Baixa,493036,3064465,Média,717989,3477466,Baixa
4,2304400,Fortaleza,CE,Nordeste,638400.0,2627482,Baixa,1023849.0,3071500,Baixa,1150975,2692722,Baixa,1219135,2841806,Baixa


In [26]:
dfish.shape

(5570, 16)

In [12]:
dfpegish = pd.merge(dfish, dfprodpegsum, left_on='Código IBGE', right_on='Cód.', how='outer')
dfpegish.head()


,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Cód.,Município_y,Pegada_Total
0,3550308,São Paulo,SP,Sudeste,10320071.0,12081082,Mínima,11771472.0,13536837,Mínima,1616686,12446934,Média,4890003,13118139,Baixa,3550308.0,São Paulo (SP),9.845771e+05
1,3304557,Rio de Janeiro,RJ,Sudeste,6267375.0,6520266,Mínima,6358858.0,6606406,Mínima,898113,6732085,Média,944945,7000802,Média,3304557.0,Rio de Janeiro (RJ),1.357514e+07
2,2927408,Salvador,BA,Nordeste,1285160.0,2953376,Baixa,1509806.0,3326869,Baixa,1518534,2893879,Baixa,1628588,3150651,Baixa,2927408.0,Salvador (BA),1.235105e+04
3,5300108,Brasília,DF,Centro-Oeste,1138566.0,2952650,Baixa,1904114.0,4111833,Baixa,493036,3064465,Média,717989,3477466,Baixa,5300108.0,Brasília (DF),1.110440e+09
4,2304400,Fortaleza,CE,Nordeste,638400.0,2627482,Baixa,1023849.0,3071500,Baixa,1150975,2692722,Baixa,1219135,2841806,Baixa,2304400.0,Fortaleza (CE),8.895986e+04


In [30]:
dfpegish.columns

Index(['Código IBGE', 'Município_x', 'UF', 'REGIAO',
       'População em Risco Total - PNSH - 2017', 'População Urbana 2017',
       'ISH Humano - PNSH - 2017', 'População em Risco Total - PNSH - 2035',
       'População Urbana 2035', 'ISH Humano - PNSH - 2035',
       'População em Risco Total - Atlas - 2020', 'População Urbana 2020',
       'ISH Humano - Atlas - 2020', 'População em Risco Total - Atlas - 2035',
       'População Urbana 2035.1', 'ISH Humano - Atlas - 2035', 'Cód.',
       'Município_y', 'Pegada_Total'],
      dtype='object')

In [59]:
dfpegish_5000.head()

,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total
377,5107925,Sorriso,MT,Centro-Oeste,11.0,74745,Máxima,35.0,109718,Máxima,1,80227,Máxima,2,93762,Máxima,8.656694e+09
136,5218805,Rio Verde,GO,Centro-Oeste,19956.0,205479,Alta,65959.0,261090,Baixa,81155,233725,Baixa,152794,355390,Baixa,5.704177e+09
799,5102637,Campo Novo do Parecis,MT,Centro-Oeste,1.0,31697,Máxima,5.0,46251,Máxima,0,33983,Máxima,0,40420,Máxima,5.378446e+09
2514,5106240,Nova Ubiratã,MT,Centro-Oeste,0.0,8385,Alta,0.0,12815,Alta,0,9072,Alta,0,10662,Alta,5.228788e+09
690,5106224,Nova Mutum,MT,Centro-Oeste,1.0,37465,Máxima,2.0,55412,Máxima,0,41203,Máxima,0,49004,Máxima,4.771651e+09


In [13]:
colunas = ['Código IBGE', 'Município_x', 'UF', 'REGIAO',
       'População em Risco Total - PNSH - 2017', 'População Urbana 2017',
       'ISH Humano - PNSH - 2017', 'População em Risco Total - PNSH - 2035',
       'População Urbana 2035', 'ISH Humano - PNSH - 2035',
       'População em Risco Total - Atlas - 2020', 'População Urbana 2020',
       'ISH Humano - Atlas - 2020', 'População em Risco Total - Atlas - 2035',
       'População Urbana 2035.1', 'ISH Humano - Atlas - 2035', 'Pegada_Total']
dfpegish = dfpegish[colunas]
dfpegish.head()

,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total
0,3550308,São Paulo,SP,Sudeste,10320071.0,12081082,Mínima,11771472.0,13536837,Mínima,1616686,12446934,Média,4890003,13118139,Baixa,9.845771e+05
1,3304557,Rio de Janeiro,RJ,Sudeste,6267375.0,6520266,Mínima,6358858.0,6606406,Mínima,898113,6732085,Média,944945,7000802,Média,1.357514e+07
2,2927408,Salvador,BA,Nordeste,1285160.0,2953376,Baixa,1509806.0,3326869,Baixa,1518534,2893879,Baixa,1628588,3150651,Baixa,1.235105e+04
3,5300108,Brasília,DF,Centro-Oeste,1138566.0,2952650,Baixa,1904114.0,4111833,Baixa,493036,3064465,Média,717989,3477466,Baixa,1.110440e+09
4,2304400,Fortaleza,CE,Nordeste,638400.0,2627482,Baixa,1023849.0,3071500,Baixa,1150975,2692722,Baixa,1219135,2841806,Baixa,8.895986e+04


In [15]:
dfpegish.shape

(5570, 17)

In [40]:
median = dfpegish['Pegada_Total'].median()
df_greater_than_median = dfpegish[dfpegish['Pegada_Total'] > median].sort_values('Pegada_Total', ascending=False)
df_greater_than_median.head()


,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total
377,5107925,Sorriso,MT,Centro-Oeste,11.0,74745,Máxima,35.0,109718,Máxima,1,80227,Máxima,2,93762,Máxima,8.656694e+09
136,5218805,Rio Verde,GO,Centro-Oeste,19956.0,205479,Alta,65959.0,261090,Baixa,81155,233725,Baixa,152794,355390,Baixa,5.704177e+09
799,5102637,Campo Novo do Parecis,MT,Centro-Oeste,1.0,31697,Máxima,5.0,46251,Máxima,0,33983,Máxima,0,40420,Máxima,5.378446e+09
2514,5106240,Nova Ubiratã,MT,Centro-Oeste,0.0,8385,Alta,0.0,12815,Alta,0,9072,Alta,0,10662,Alta,5.228788e+09
690,5106224,Nova Mutum,MT,Centro-Oeste,1.0,37465,Máxima,2.0,55412,Máxima,0,41203,Máxima,0,49004,Máxima,4.771651e+09


In [41]:
df_greater_than_median.shape

(2749, 17)

In [42]:
# Create a boxplot using Altair
boxplot = alt.Chart(df_greater_than_median).mark_boxplot(extent='min-max').encode(
    x='Pegada_Total',
    y='REGIAO'
)

# Display the boxplot
boxplot

alt.Chart(...)

In [43]:
mean = dfpegish['Pegada_Total'].mean()
df_greater_than_mean = dfpegish[dfpegish['Pegada_Total'] > mean].sort_values('Pegada_Total', ascending=False)
df_greater_than_mean.head()


,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total
377,5107925,Sorriso,MT,Centro-Oeste,11.0,74745,Máxima,35.0,109718,Máxima,1,80227,Máxima,2,93762,Máxima,8.656694e+09
136,5218805,Rio Verde,GO,Centro-Oeste,19956.0,205479,Alta,65959.0,261090,Baixa,81155,233725,Baixa,152794,355390,Baixa,5.704177e+09
799,5102637,Campo Novo do Parecis,MT,Centro-Oeste,1.0,31697,Máxima,5.0,46251,Máxima,0,33983,Máxima,0,40420,Máxima,5.378446e+09
2514,5106240,Nova Ubiratã,MT,Centro-Oeste,0.0,8385,Alta,0.0,12815,Alta,0,9072,Alta,0,10662,Alta,5.228788e+09
690,5106224,Nova Mutum,MT,Centro-Oeste,1.0,37465,Máxima,2.0,55412,Máxima,0,41203,Máxima,0,49004,Máxima,4.771651e+09


In [44]:
df_greater_than_mean.shape

(1150, 17)

In [49]:
# Create a boxplot using Altair
boxplot = alt.Chart(df_greater_than_mean).mark_boxplot(extent='min-max').encode(
    x='Pegada_Total',
    y='REGIAO'
).properties(
    width=1200,
    height=420
).interactive()

# Display the boxplot
boxplot

alt.Chart(...)

In [14]:
# Calculate the total sum of 'Pegada_Total'
total_sum = dfpegish['Pegada_Total'].sum()

# Sort the dataframe by 'Pegada_Total' in descending order
sorted_df = dfpegish.sort_values('Pegada_Total', ascending=False)

# Calculate the cumulative sum of 'Pegada_Total'
sorted_df['Cumulative_Sum'] = sorted_df['Pegada_Total'].cumsum()

# Filter the dataframe to include only the rows where the cumulative sum is less than or equal to 50% of the total sum
filtered_df = sorted_df[sorted_df['Cumulative_Sum'] <= 0.50 * total_sum]

# Display the filtered dataframe
filtered_df.shape


(232, 18)

In [79]:
filtered_df[filtered_df['ISH Humano - PNSH - 2017'].isin(['Mínima', 'Baixa'])]


,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total,Cumulative_Sum
3,5300108,Brasília,DF,Centro-Oeste,1138566.0,2952650,Baixa,1904114.0,4111833,Baixa,493036,3064465,Média,717989,3477466,Baixa,1.110440e+09,1.720995e+11
225,4322400,Uruguaiana,RS,Sul,111099.0,121721,Mínima,93448.0,102055,Mínima,0,119582,Máxima,0,121333,Máxima,9.070590e+08,1.937249e+11
708,4306601,Dom Pedrito,RS,Sul,20856.0,36554,Baixa,15664.0,30750,Baixa,11252,35494,Média,11279,35006,Média,8.932339e+08,1.955240e+11
693,1506708,Santana do Araguaia,PA,Norte,17525.0,37364,Baixa,46439.0,66806,Baixa,2,38850,Alta,3,42516,Alta,8.822196e+08,1.972916e+11
753,4310603,Itaqui,RS,Sul,31196.0,33983,Mínima,20717.0,28700,Baixa,0,32747,Máxima,0,31480,Máxima,7.948881e+08,2.083209e+11
510,4303509,Camaquã,RS,Sul,37835.0,52397,Baixa,47734.0,60268,Baixa,29377,52271,Baixa,32309,52321,Baixa,6.307124e+08,2.336909e+11
1068,2302206,Beberibe,CE,Nordeste,23263.0,23263,Mínima,30609.0,30609,Mínima,129,23851,Alta,152,25205,Alta,5.999969e+08,2.385870e+11
640,5213806,Morrinhos,GO,Centro-Oeste,19322.0,40275,Baixa,34727.0,52700,Baixa,16154,41531,Média,20639,44501,Baixa,5.459378e+08,2.482526e+11
153,2918407,Juazeiro,BA,Nordeste,186673.0,187529,Mínima,210620.0,211339,Mínima,0,188030,Máxima,0,222230,Máxima,5.296981e+08,2.536486e+11
1324,5002209,Bonito,MS,Centro-Oeste,14873.0,18561,Mínima,18880.0,22568,Mínima,1,19598,Máxima,1,22657,Máxima,5.172891e+08,2.572941e+11


In [80]:
filtered_df[filtered_df['ISH Humano - PNSH - 2035'].isin(['Mínima', 'Baixa'])]


,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total,Cumulative_Sum
136,5218805,Rio Verde,GO,Centro-Oeste,19956.0,205479,Alta,65959.0,261090,Baixa,81155,233725,Baixa,152794,355390,Baixa,5.704177e+09,1.436087e+10
3,5300108,Brasília,DF,Centro-Oeste,1138566.0,2952650,Baixa,1904114.0,4111833,Baixa,493036,3064465,Média,717989,3477466,Baixa,1.110440e+09,1.720995e+11
602,5218508,Quirinópolis,GO,Centro-Oeste,8160.0,43919,Alta,36340.0,57003,Baixa,165,46235,Máxima,231,51654,Máxima,1.063472e+09,1.753106e+11
131,5107602,Rondonópolis,MT,Centro-Oeste,42449.0,217336,Alta,70418.0,266638,Baixa,37,232380,Máxima,62,293535,Máxima,9.805121e+08,1.824342e+11
84,4104808,Cascavel,PR,Sul,53945.0,305467,Média,68909.0,327578,Baixa,72350,320812,Baixa,31313,359158,Alta,9.100182e+08,1.928178e+11
225,4322400,Uruguaiana,RS,Sul,111099.0,121721,Mínima,93448.0,102055,Mínima,0,119582,Máxima,0,121333,Máxima,9.070590e+08,1.937249e+11
708,4306601,Dom Pedrito,RS,Sul,20856.0,36554,Baixa,15664.0,30750,Baixa,11252,35494,Média,11279,35006,Média,8.932339e+08,1.955240e+11
693,1506708,Santana do Araguaia,PA,Norte,17525.0,37364,Baixa,46439.0,66806,Baixa,2,38850,Alta,3,42516,Alta,8.822196e+08,1.972916e+11
753,4310603,Itaqui,RS,Sul,31196.0,33983,Mínima,20717.0,28700,Baixa,0,32747,Máxima,0,31480,Máxima,7.948881e+08,2.083209e+11
520,3127107,Frutal,MG,Sudeste,13398.0,51618,Média,32820.0,64215,Baixa,27721,53316,Baixa,35518,57270,Baixa,7.657013e+08,2.122188e+11


In [81]:
filtered_df[filtered_df['ISH Humano - Atlas - 2020'].isin(['Mínima', 'Baixa'])]


,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total,Cumulative_Sum
136,5218805,Rio Verde,GO,Centro-Oeste,19956.0,205479,Alta,65959.0,261090,Baixa,81155,233725,Baixa,152794,355390,Baixa,5.704177e+09,1.436087e+10
2497,5216403,Paraúna,GO,Centro-Oeste,698.0,8481,Máxima,4110.0,11952,Alta,6719,8453,Baixa,7594,9325,Baixa,1.791077e+09,1.270705e+11
291,5205109,Catalão,GO,Centro-Oeste,11132.0,96755,Alta,24107.0,122566,Alta,45874,105694,Baixa,61673,134600,Baixa,1.126172e+09,1.698726e+11
84,4104808,Cascavel,PR,Sul,53945.0,305467,Média,68909.0,327578,Baixa,72350,320812,Baixa,31313,359158,Alta,9.100182e+08,1.928178e+11
520,3127107,Frutal,MG,Sudeste,13398.0,51618,Média,32820.0,64215,Baixa,27721,53316,Baixa,35518,57270,Baixa,7.657013e+08,2.122188e+11
510,4303509,Camaquã,RS,Sul,37835.0,52397,Baixa,47734.0,60268,Baixa,29377,52271,Baixa,32309,52321,Baixa,6.307124e+08,2.336909e+11
439,4306106,Cruz Alta,RS,Sul,41151.0,62647,Baixa,39531.0,60098,Baixa,24075,59687,Baixa,16928,55495,Média,4.940218e+08,2.628490e+11
1842,5105903,Nobres,MT,Centro-Oeste,5043.0,12706,Média,12524.0,18830,Baixa,11347,13196,Mínima,12024,13601,Mínima,4.870707e+08,2.643143e+11
634,3136306,João Pinheiro,MG,Sudeste,5541.0,40586,Média,20909.0,45776,Baixa,25868,41704,Baixa,31880,44660,Baixa,4.683305e+08,2.700544e+11
820,3513702,Descalvado,SP,Sudeste,6817.0,30671,Média,11166.0,36616,Média,13265,31843,Baixa,14668,34112,Baixa,4.523249e+08,2.751007e+11


In [82]:
filtered_df[filtered_df['ISH Humano - Atlas - 2035'].isin(['Mínima', 'Baixa'])]


,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total,Cumulative_Sum
136,5218805,Rio Verde,GO,Centro-Oeste,19956.0,205479,Alta,65959.0,261090,Baixa,81155,233725,Baixa,152794,355390,Baixa,5.704177e+09,1.436087e+10
2497,5216403,Paraúna,GO,Centro-Oeste,698.0,8481,Máxima,4110.0,11952,Alta,6719,8453,Baixa,7594,9325,Baixa,1.791077e+09,1.270705e+11
291,5205109,Catalão,GO,Centro-Oeste,11132.0,96755,Alta,24107.0,122566,Alta,45874,105694,Baixa,61673,134600,Baixa,1.126172e+09,1.698726e+11
3,5300108,Brasília,DF,Centro-Oeste,1138566.0,2952650,Baixa,1904114.0,4111833,Baixa,493036,3064465,Média,717989,3477466,Baixa,1.110440e+09,1.720995e+11
520,3127107,Frutal,MG,Sudeste,13398.0,51618,Média,32820.0,64215,Baixa,27721,53316,Baixa,35518,57270,Baixa,7.657013e+08,2.122188e+11
510,4303509,Camaquã,RS,Sul,37835.0,52397,Baixa,47734.0,60268,Baixa,29377,52271,Baixa,32309,52321,Baixa,6.307124e+08,2.336909e+11
640,5213806,Morrinhos,GO,Centro-Oeste,19322.0,40275,Baixa,34727.0,52700,Baixa,16154,41531,Média,20639,44501,Baixa,5.459378e+08,2.482526e+11
1842,5105903,Nobres,MT,Centro-Oeste,5043.0,12706,Média,12524.0,18830,Baixa,11347,13196,Mínima,12024,13601,Mínima,4.870707e+08,2.643143e+11
634,3136306,João Pinheiro,MG,Sudeste,5541.0,40586,Média,20909.0,45776,Baixa,25868,41704,Baixa,31880,44660,Baixa,4.683305e+08,2.700544e+11
614,3143104,Monte Carmelo,MG,Sudeste,26044.0,42443,Baixa,42530.0,52528,Mínima,17035,42714,Média,21779,48225,Baixa,4.648630e+08,2.709875e+11


In [64]:
# Sort the dataframe by 'Pegada_Total' in descending order
sorted_df = filtered_df.sort_values('Pegada_Total', ascending=False)

# Create a boxplot using Altair
boxplot = alt.Chart(sorted_df).mark_boxplot(extent='min-max').encode(
    y=alt.Y('REGIAO:N', sort='-x'),
    x='Pegada_Total'
).properties(
    width=800,
    height=600
).interactive()

# Display the boxplot
boxplot


alt.Chart(...)

In [60]:
dfpegish_grouped = dfpegish.groupby('REGIAO')['Pegada_Total'].sum()
dfpegish_grouped


REGIAO
Centro-Oeste    2.146833e+11
Nordeste        6.463818e+10
Norte           3.335587e+10
Sudeste         1.309427e+11
Sul             1.108286e+11
Name: Pegada_Total, dtype: float64

In [63]:
# Sort the dataframe by 'Pegada_Total' in descending order
dfpegish_grouped_sorted = dfpegish_grouped.sort_values(ascending=False)

# Create a bar chart using Altair
bar_chart = alt.Chart(dfpegish_grouped_sorted.reset_index()).mark_bar().encode(
    x=alt.X('REGIAO:N', sort='-y'),
    y='Pegada_Total:Q'
).properties(
    width=800,
    height=600
).interactive()

# Display the bar chart
bar_chart


alt.Chart(...)

## Biomas

In [16]:
# Specify the file path
file_path = 'Lista_Municipio_Bioma_250mil.xls'

# Load the Excel file into a pandas DataFrame
df_bioma = pd.read_excel(file_path)

# Display the DataFrame
df_bioma.head()


,CD_GEOCMU,NM_MUNICIP,BIOMA
0,5200050,ABADIA DE GOIÁS,Cerrado
1,3100104,ABADIA DOS DOURADOS,Cerrado
2,5200100,ABADIÂNIA,Cerrado
3,1500107,ABAETETUBA,Amazônia
4,3100203,ABAETÉ,Cerrado


In [24]:
df_bioma.drop_duplicates(inplace=True)
df_bioma.shape

(6539, 3)

In [18]:
dfpegish.head()

,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total
0,3550308,São Paulo,SP,Sudeste,10320071.0,12081082,Mínima,11771472.0,13536837,Mínima,1616686,12446934,Média,4890003,13118139,Baixa,9.845771e+05
1,3304557,Rio de Janeiro,RJ,Sudeste,6267375.0,6520266,Mínima,6358858.0,6606406,Mínima,898113,6732085,Média,944945,7000802,Média,1.357514e+07
2,2927408,Salvador,BA,Nordeste,1285160.0,2953376,Baixa,1509806.0,3326869,Baixa,1518534,2893879,Baixa,1628588,3150651,Baixa,1.235105e+04
3,5300108,Brasília,DF,Centro-Oeste,1138566.0,2952650,Baixa,1904114.0,4111833,Baixa,493036,3064465,Média,717989,3477466,Baixa,1.110440e+09
4,2304400,Fortaleza,CE,Nordeste,638400.0,2627482,Baixa,1023849.0,3071500,Baixa,1150975,2692722,Baixa,1219135,2841806,Baixa,8.895986e+04


In [30]:
dfpegishbioma = pd.merge(dfpegish, df_bioma, left_on='Código IBGE', right_on='CD_GEOCMU', how='outer')
dfpegishbioma.shape

(6539, 20)

In [31]:
dfpegishbioma.head()

,Código IBGE,Município_x,UF,REGIAO,População em Risco Total - PNSH - 2017,População Urbana 2017,ISH Humano - PNSH - 2017,População em Risco Total - PNSH - 2035,População Urbana 2035,ISH Humano - PNSH - 2035,População em Risco Total - Atlas - 2020,População Urbana 2020,ISH Humano - Atlas - 2020,População em Risco Total - Atlas - 2035,População Urbana 2035.1,ISH Humano - Atlas - 2035,Pegada_Total,CD_GEOCMU,NM_MUNICIP,BIOMA
0,3550308.0,São Paulo,SP,Sudeste,10320071.0,12081082.0,Mínima,11771472.0,13536837.0,Mínima,1616686.0,12446934.0,Média,4890003.0,13118139.0,Baixa,9.845771e+05,3550308,SÃO PAULO,Mata Atlântica
1,3304557.0,Rio de Janeiro,RJ,Sudeste,6267375.0,6520266.0,Mínima,6358858.0,6606406.0,Mínima,898113.0,6732085.0,Média,944945.0,7000802.0,Média,1.357514e+07,3304557,RIO DE JANEIRO,Mata Atlântica
2,2927408.0,Salvador,BA,Nordeste,1285160.0,2953376.0,Baixa,1509806.0,3326869.0,Baixa,1518534.0,2893879.0,Baixa,1628588.0,3150651.0,Baixa,1.235105e+04,2927408,SALVADOR,Mata Atlântica
3,5300108.0,Brasília,DF,Centro-Oeste,1138566.0,2952650.0,Baixa,1904114.0,4111833.0,Baixa,493036.0,3064465.0,Média,717989.0,3477466.0,Baixa,1.110440e+09,5300108,BRASÍLIA,Cerrado
4,2304400.0,Fortaleza,CE,Nordeste,638400.0,2627482.0,Baixa,1023849.0,3071500.0,Baixa,1150975.0,2692722.0,Baixa,1219135.0,2841806.0,Baixa,8.895986e+04,2304400,FORTALEZA,Caatinga


In [35]:
dfpegishbioma.columns

Index(['Código IBGE', 'Município_x', 'UF', 'REGIAO',
       'População em Risco Total - PNSH - 2017', 'População Urbana 2017',
       'ISH Humano - PNSH - 2017', 'População em Risco Total - PNSH - 2035',
       'População Urbana 2035', 'ISH Humano - PNSH - 2035',
       'População em Risco Total - Atlas - 2020', 'População Urbana 2020',
       'ISH Humano - Atlas - 2020', 'População em Risco Total - Atlas - 2035',
       'População Urbana 2035.1', 'ISH Humano - Atlas - 2035', 'Pegada_Total',
       'CD_GEOCMU', 'NM_MUNICIP', 'BIOMA'],
      dtype='object')

#### ISH Atlas 2020

In [91]:
df_grouped = dfpegishbioma.groupby(['BIOMA', 'ISH Humano - Atlas - 2020']).size().reset_index(name='Count')
df_grouped = df_grouped[df_grouped['ISH Humano - Atlas - 2020'] != 0]

value_mapping = {'Mínima': '1. Mínima', 'Baixa': '2. Baixa', 'Média': '3. Média', 'Alta': '4. Alta', 'Máxima': '5. Máxima'}
df_grouped['New_Column'] = df_grouped['ISH Humano - Atlas - 2020'].map(value_mapping)
df_grouped.head()

,BIOMA,ISH Humano - Atlas - 2020,Count,New_Column
1,Amazônia,Alta,320,4. Alta
2,Amazônia,Baixa,9,2. Baixa
3,Amazônia,Máxima,209,5. Máxima
4,Amazônia,Média,15,3. Média
5,Amazônia,Mínima,4,1. Mínima


In [92]:
import altair as alt

# Create a stacked bar chart using Altair
bar_chart = alt.Chart(df_grouped).mark_bar().encode(
    x='BIOMA:N',
    y=alt.Y('Count:Q', stack='normalize', sort='descending'),
    color=alt.Color('New_Column:N', legend=alt.Legend(title="ISH Humano - Atlas - 2020")),
    tooltip=['BIOMA', 'Count', 'ISH Humano - Atlas - 2020'],
).properties(
    width=800,
    height=600
).interactive()

# Display the bar chart
bar_chart


alt.Chart(...)

### ISH Atlas 2035

In [94]:
df_grouped2035 = dfpegishbioma.groupby(['BIOMA', 'ISH Humano - Atlas - 2035']).size().reset_index(name='Count')
df_grouped2035 = df_grouped2035[df_grouped2035['ISH Humano - Atlas - 2035'] != 0]

value_mapping = {'Mínima': '1. Mínima', 'Baixa': '2. Baixa', 'Média': '3. Média', 'Alta': '4. Alta', 'Máxima': '5. Máxima'}
df_grouped2035['New_Column'] = df_grouped2035['ISH Humano - Atlas - 2035'].map(value_mapping)

# Create a stacked bar chart using Altair
bar_chart = alt.Chart(df_grouped2035).mark_bar().encode(
    x='BIOMA:N',
    y=alt.Y('Count:Q', stack='normalize', sort='descending'),
    color=alt.Color('New_Column:N', legend=alt.Legend(title="ISH Humano - Atlas - 2035")),
    tooltip=['BIOMA', 'Count', 'ISH Humano - Atlas - 2035'],
).properties(
    width=800,
    height=600
)

# Display the bar chart
bar_chart


alt.Chart(...)

### Agora recortando pelas maiores pegadas

In [95]:
# Calculate the total sum of 'Pegada_Total'
total_sum = dfpegish['Pegada_Total'].sum()

# Sort the dataframe by 'Pegada_Total' in descending order
sorted_df = dfpegish.sort_values('Pegada_Total', ascending=False)

# Calculate the cumulative sum of 'Pegada_Total'
sorted_df['Cumulative_Sum'] = sorted_df['Pegada_Total'].cumsum()

# Filter the dataframe to include only the rows where the cumulative sum is less than or equal to 50% of the total sum
filtered_df = sorted_df[sorted_df['Cumulative_Sum'] <= 0.50 * total_sum]

# Display the filtered dataframe
filtered_df.shape


(232, 18)

In [96]:
df50_bioma = pd.merge(filtered_df, df_bioma, left_on='Código IBGE', right_on='CD_GEOCMU', how='inner')
df50_bioma.shape

(318, 21)

### ISH Atlas 2020

In [98]:
df_grouped = df50_bioma.groupby(['BIOMA', 'ISH Humano - Atlas - 2020']).size().reset_index(name='Count')
df_grouped = df_grouped[df_grouped['ISH Humano - Atlas - 2020'] != 0]

value_mapping = {'Mínima': '1. Mínima', 'Baixa': '2. Baixa', 'Média': '3. Média', 'Alta': '4. Alta', 'Máxima': '5. Máxima'}
df_grouped['New_Column'] = df_grouped['ISH Humano - Atlas - 2020'].map(value_mapping)
df_grouped.head()

,BIOMA,ISH Humano - Atlas - 2020,Count,New_Column
0,Amazônia,Alta,1,4. Alta
1,Amazônia,Baixa,1,2. Baixa
2,Amazônia,Máxima,1,5. Máxima
3,Amazônia,Média,1,3. Média
4,Amazônia,Mínima,1,1. Mínima
